In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import numpy.random as rand
from sklearn.metrics import silhouette_score
import copy
import pandas as pd
from numpy.random import randint
from numpy.random import rand
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import homogeneity_score
from sklearn.metrics.cluster import rand_score
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import completeness_score
from sklearn.metrics import f1_score
from csv import writer

In [ ]:
df= np.load(r"/content/gdrive/My Drive/PHD/JAYPEE/Embed_NEW/SLF/output/highland_outward.npy")

In [ ]:
df.shape

(17, 32)

In [ ]:
def K_means_clustering(datapoint,s):
    a=s
    v=np.array(a)
    x=v.reshape(3,32)
    km=KMeans(n_clusters=3,init=x, n_init=1)
    km.fit(datapoint)
    score = silhouette_score(datapoint, km.labels_, metric='euclidean')
    def ClusterIndicesNumpy(clustNum, labels_array):
        return np.where(labels_array == clustNum)[0]
    cluster_1_records=datapoint[ClusterIndicesNumpy(0,km.labels_)]
    cluster_2_records=datapoint[ClusterIndicesNumpy(1,km.labels_)]
    cluster_3_records=datapoint[ClusterIndicesNumpy(2,km.labels_)]
    return [cluster_1_records,cluster_2_records,cluster_3_records]

In [ ]:
df=np.nan_to_num(df, nan=0)

In [ ]:
def fitnes_function(position):
    x,y,z=K_means_clustering(df,position)
    fitness_value_x=intra_cluster(x)
    fitness_value_y=intra_cluster(y)
    fitness_value_z=intra_cluster(z)
    fitness_value=float(np.asarray(fitness_value_x))+float(np.asarray(fitness_value_y))+float(np.asarray(fitness_value_z))
    return fitness_value

In [ ]:
#pos:mean of the cluster
def intra_cluster(data_point):
    import numpy as np
    c=data_point
    #print("shape of data vector is",c.shape)
    e=c.transpose()
    #print(e.shape)
    m,n=e.shape
    s1=0
    #mean_cluster=0
    total_sum_data_vector=0
    for x in range(0,n):
        p=c[:][x]
        #print(p.shape)
        data_vector=p.reshape(m,1)
        #print("shape of data point of cluster",data_vector.shape)
        total_sum_data_vector+=data_vector
        #print("total sum of individual data vectors",total_sum_data_vector)
        #print("shape of total sum of individual data vectors",total_sum_data_vector.shape)
        mean_cluster=total_sum_data_vector/n
        #print("mean of each cluster",mean_cluster)
    for x in range(0,n):
        p=c[:][x]
        #print(p.shape)
        data_vector=p.reshape(m,1)
        #print("shape of data point of cluster",data_vector.shape)
        diff=(data_vector-mean_cluster)
        #print("difference of data point with cluster difference",diff.shape)
        s1+=np.dot(diff,diff.transpose())
        #print(s1)
   
    within=s1.trace()
    intra=within.reshape(1,1)
    ## To change
    inter=inter_cluster(data_point)
    res=within/inter
    return [float(np.asarray(res))]

In [ ]:
def inter_cluster(data_point):
    total_sum_data_vector=0
    c=data_point
    s1=0
    m,n=c.shape
    f=c.transpose()
    for x in range(0,n):
        p=f[:][x]
        data_vector=p.reshape(m,1)
        total_sum_data_vector+=data_vector
        mean_cluster=total_sum_data_vector/n
        mean_overall_cluster=data_point/m*n
        diff=(mean_cluster-mean_overall_cluster)
        s1+=np.dot(diff,diff.transpose())
    inter=s1.trace()
    inter.reshape(1,1)
    
    return[float(np.asarray(inter))]

In [ ]:
# tournament selection- parents are selected
def selection(pop, scores, k=3):
	# first random selection
	selection_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		# check if better (e.g. perform a tournament)
		if scores[ix] < scores[selection_ix]:
			selection_ix = ix
			#print(scores[ix])
	return pop[selection_ix]

In [ ]:
# mutation operator-change
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# check for a mutation
		if rand() < r_mut:
			# flip the bit
			bitstring[i] = 1 - bitstring[i]

In [ ]:
# crossover two parents to create two children
def crossover(p1, p2, P):
	c1, c2 = p1.flatten(), p2.flatten()
	#c1, c2 = p1.copy(), p2.copy()
  
  
	for i in range(len(P)):
		if(P[i]<0.5):
			temp=c1[i]
			c1[i]=c2[i]
			c2[i]=temp
	c1,c2=c1.reshape(3,32),c2.reshape(3,32)
  
  
  
	return [c1, c2]

In [ ]:
n_pop=50

In [ ]:
Arr=np.zeros(df.shape[0])
Highland1=[0,1,12,14]
Highland2=[13,8,9,12,4]
Highland3=[6,7,3,2,5,11,10]
for i in range(df.shape[0]):
   if(i in Highland1):
     Arr[i]=1;
   elif(i in Highland2):
     Arr[i]=2;
   elif(i in Highland3):
     Arr[i]=3;
df2=pd.DataFrame(Arr);

n_pop=10
n_iter=5
r_cross = 0.9
r_mut=0.1

pop = [df[np.random.randint(1,16,size=3)] for _ in range(n_pop)]

best, best_eval = 0, fitnes_function(pop[0])
for gen in range(n_iter):
		scores = [fitnes_function(c) for c in pop]
		
		for i in range(n_pop):
			if scores[i] < best_eval:
				best, best_eval = pop[i], scores[i]
				#print(">%d, new best f(%s) = %.3f" % (gen,  pop[i], scores[i]))
		
		# select parents
		selected = [selection(pop, scores) for _ in range(n_pop)]
		
children = list()
for i in range(0, n_pop, 2):
			# get selected parents in pairs
			p1, p2 = selected[i], selected[i+1]
			# crossover and mutation
			p=np.random.rand(10)
			for c in crossover(p1, p2, p):
				# mutation
				mutation(c, r_mut)
				# store for next generation
				#score1 = fitnes_function(c)
				print("p1--",p1,"   p2---",p2,"   c---",c)

for i in children:
	v=np.array(i)
  x=v.reshape(3,32)
  km=KMeans(n_clusters=3,init=x, n_init=1)
  d=km.fit_predict(df)
  df1 = pd.DataFrame(d);
  df = pd.DataFrame(df);
  df["Predicted"]=df1.values
  df["Actual"]=df2.values
  

  List=["after",completeness_score(d,Arr),homogeneity_score(d, Arr),rand_score(d, Arr),normalized_mutual_info_score(d, Arr)]
  with open('GGWWOO.csv', 'a') as f_object:

      writer_object = writer(f_object)
      writer_object.writerow(List)
      f_object.close()



p1-- [[0.16642813 0.98685231 0.94968437 0.35430921 0.64673517 0.74530344
  0.25090921 0.73554195 0.63796708 0.79971225 0.1960021  0.33273563
  0.92664842 0.25227415 0.82902224 0.85345434 0.08714022 0.27379945
  0.37139383 0.91861841 0.83927374 0.45631206 0.99374588 0.92685596
  0.62948137 0.85847234 1.47317704 0.55244089 0.42478917 1.10884079
  0.         0.71769036]
 [0.27029619 0.41543256 0.41106942 0.00667286 0.1105772  0.
  0.816539   0.63141008 0.03064739 0.23268701 0.12626809 0.45561537
  0.08351706 0.56911285 0.08931914 0.86037218 0.34635317 1.01657008
  0.23145264 0.23483359 0.23161114 0.95459035 0.82592835 0.55570052
  0.68314828 0.05429004 0.51722794 0.94720569 1.01109672 0.09815332
  1.0005002  0.07464351]
 [0.50428645 0.10855985 0.85212036 0.02799493 0.88425562 0.21396315
  0.83720358 0.39056496 0.77161457 0.70125764 0.71134241 0.67545514
  0.61222176 0.0636121  0.08991507 0.97938599 0.13728242 0.25718669
  1.03475072 0.36834121 0.94093027 0.25166165 0.57144267 0.69901081
 